In [ ]:
!git clone https://github.com/junyanz/pytorch-CycleGAN-and-pix2pix

Cloning into 'pytorch-CycleGAN-and-pix2pix'...
remote: Enumerating objects: 2516, done.
remote: Total 2516 (delta 0), reused 0 (delta 0), pack-reused 2516 (from 1)
Receiving objects: 100% (2516/2516), 8.20 MiB | 9.03 MiB/s, done.
Resolving deltas: 100% (1575/1575), done.


In [ ]:
import os
os.chdir('pytorch-CycleGAN-and-pix2pix/')

In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 26.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.4/311.4 kB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.8 MB/s eta 0:00:00
  Created wheel for visdom: filename=visdom-0.2.4-py3-none-any.whl size=1408195 sha256=d9a6e82843576196b0f846e7891e5e89de601d5b049c04152b79cb91f587c31a
  Stored in directory: /root/.cache/pip/wheels/42/29/49/5bed207bac4578e4d2c0c5fc0226bfd33a7e2953ea56356855
Successfully built visdom


In [ ]:
!bash ./scripts/download_pix2pix_model.sh edges2shoes

Note: available models are edges2shoes, sat2map, map2sat, facades_label2photo, and day2night
Specified [edges2shoes]
for details.

--2024-09-29 06:12:52--  http://efrosgans.eecs.berkeley.edu/pix2pix/models-pytorch/edges2shoes.pth
Resolving efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)... 128.32.244.190
Connecting to efrosgans.eecs.berkeley.edu (efrosgans.eecs.berkeley.edu)|128.32.244.190|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 217704688 (208M)
Saving to: ‘./checkpoints/edges2shoes_pretrained/latest_net_G.pth’

./checkpoints/edges 100%[===================>] 207.62M  45.2MB/s    in 5.0s    

2024-09-29 06:12:57 (41.7 MB/s) - ‘./checkpoints/edges2shoes_pretrained/latest_net_G.pth’ saved [217704688/217704688]



In [ ]:
import torch
from torchvision import transforms
from torchvision.utils import save_image
from PIL import Image
import os
from models.networks import define_G  # Assuming this module is available

# Load Pretrained Pix2Pix model
def load_pix2pix_model(model_path):
    model = define_G(3, 3, 64, 'unet_256', norm='batch', use_dropout=True)
    model.load_state_dict(torch.load(model_path))
    model.eval()  # Set model to evaluation mode
    return model

# Image transformation (normalization as per training setup)
def get_transforms():
    return transforms.Compose([
        transforms.Resize((256, 256)),  # Resizing the image to match the model input size
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalizing between -1 and 1
    ])

# Function to generate and save output image
def generate_and_save_image(input_image_path, model, save_path):
    # Load the input image
    input_image = Image.open('/content/drive/MyDrive/Colab Notebooks/Mikey test.jpg').convert('RGB')

    # Transform the image
    transform = get_transforms()
    transformed_image = transform(input_image).unsqueeze(0).cuda()  # Add batch dimension and move to GPU

    # Generate the output
    with torch.no_grad():  # No need to compute gradients for inference
        output_image = model(transformed_image)

    output_image = (output_image * 0.5) + 0.5  # Reverse normalization to [0, 1]

    # Ensure the save_path has a proper extension
    if not save_path.endswith('.png') and not save_path.endswith('.jpg'):
        save_path += '.png'  # Default to PNG if no extension is provided

    save_image(output_image, save_path)
    print(f"Generated image saved at: {'/content/pytorch-CycleGAN-and-pix2pix/imgs'}")

# Load the pre-trained model (ensure the path is correct)
model_path = './checkpoints/edges2shoes_pretrained/latest_net_G.pth'
generator = load_pix2pix_model(model_path).cuda()

# Example usage: translate input image and save output
input_image_path = '/content/drive/MyDrive/Colab Notebooks/Mikey test.jpg'  # Provide the path to your test image
output_image_path = '/content/pytorch-CycleGAN-and-pix2pix/imgs'  # Path where the output image will be saved

# Generate and save the translated image
generate_and_save_image(input_image_path, generator, output_image_path)


initialize network with normal


<ipython-input-7-03ec6737283d>:11: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Generated image saved at: /content/pytorch-CycleGAN-and-pix2pix/imgs
